In [1]:
import pandas
import numpy
import glob
import os
import re

In [2]:
schedule_files = glob.glob(os.path.expanduser('~/Dropbox/Uncertain Principles/Articles/NCAAWomen2019/data/team_schedules/*csv'))

In [3]:
def get_other(s):
    m = re.search('^\s*@?\s*(\S.*?)@', s)
    if m is not None:
        other = m.groups()[0].strip()
    else:
        m = re.search('^\s*@(.*)', s)
        if m is not None:
            other = m.groups()[0].strip()
        else:
            other = s.strip()
        
    return other

In [4]:
def get_id(name, teams):
    name = re.sub(' ', '_', name)
    id_list = list(teams[teams['name'] == name]['id'])
    if len(id_list) == 0:
        name_id = -1
    elif len(id_list) > 1:
        raise ValueError('ERROR - non-unique name: {} --> {}'.format(name, id_list))
    else:
        name_id = int(id_list[0])
        
    return int(name_id)

In [5]:
def parse_score(score_str, opponent='team'):
    m = re.search('[WL]?\s*(\d+)\s*-\s*(\d+)', score_str)
    if m is not None:
        team_score, other_score = map(int, m.groups())
    else:
        raise ValueError('ERROR - malformed score: {}'.format(score_str))
        
    if opponent == 'team':
        return team_score
    elif opponent == 'other':
        return other_score
    else:
        raise ValueError('ERROR - malformed opponent, must be team or other but received: {}'.format(opponent))

In [6]:
def valid_score(score_str):
    m = re.search('[WL]?\s*(\d+)\s*-\s*(\d+)', score_str)
    if m is not None:
        valid_score = True
    else:
        valid_score = False
        
    return valid_score

In [7]:
def team_is_home(team, opponent):
    if re.search('\S+\s*@', opponent) is not None:
        # Uh oh - neutral location with no home team
        team_hash = hash(team)
        opponent_hash = hash(get_other(opponent))
        if team_hash < opponent_hash:
            team_is_home = True
        else:
            team_is_home = False
    elif re.search('^\s*@', opponent) is None:
        # no leading @ so team is home
        team_is_home = True
    else:
        # leading @ so team is guest
        team_is_home = False
        
    return team_is_home

In [8]:
def format_team_data(team_data, teams):
    team_data = team_data.rename(index=str, columns={'teamname': 'team',
                                                     'home_points': 'team_points',
                                                    })
    
    team = list(team_data['team'].drop_duplicates())[0]
    
    team_data['valid_score'] = team_data['result'].apply(valid_score)
    team_data = team_data[team_data['valid_score'] == True]

    team_data['team_id'] = team_data['team'].apply(lambda o_name: get_id(o_name, teams))
    team_data['team_score'] = team_data['result'].apply(lambda s: parse_score(s, 'team'))
    team_data['other'] = team_data['guest'].apply(get_other)
    team_data['other_id'] = team_data['other'].apply(lambda o_name: get_id(o_name, teams))
    team_data['other_score'] = team_data['result'].apply(lambda s: parse_score(s, 'other'))
    team_data['team_is_home'] = team_data['guest'].apply(lambda s: team_is_home(team, s))
    team_data['neutral_location'] = team_data['guest'].apply(lambda s: re.search('\S+\s*@', s) is not None)
        
    team_data = team_data[(team_data['team_id'] != -1) & (team_data['other_id'] != -1)]
    team_data.fillna(value={'3FG': 0, '3FGA': 0, 'FGM': 0, 'FGA': 0, 'FT':0, 'FTA':0}, inplace=True)

    team_data['home'] = numpy.where(team_data['team_is_home'] == True, team_data['team'], team_data['other'])
    team_data['home_id'] = numpy.where(team_data['team_is_home'] == True, team_data['team_id'], team_data['other_id'])
    team_data['home_score'] = numpy.where(team_data['team_is_home'] == True, team_data['team_score'], team_data['other_score'])
    team_data['guest'] = numpy.where(team_data['team_is_home'] == False, team_data['team'], team_data['other'])
    team_data['guest_id'] = numpy.where(team_data['team_is_home'] == False, team_data['team_id'], team_data['other_id'])
    team_data['guest_score'] = numpy.where(team_data['team_is_home'] == False, team_data['team_score'], team_data['other_score'])
    team_data['spread'] = team_data['home_score'] - team_data['guest_score']

    team_data['game_id'] = team_data['home_id'].map(str) + '/' + \
                           team_data['guest_id'].map(str)+ '/' + \
                           team_data['gamedate'].map(str)
    
    team_data['home_3FG']  = numpy.where(team_data['team_is_home'] == True, team_data['3FG'], 0)
    team_data['home_3FGA'] = numpy.where(team_data['team_is_home'] == True, team_data['3FGA'], 0)
    team_data['home_FGM']  = numpy.where(team_data['team_is_home'] == True, team_data['FGM'], 0)
    team_data['home_FGA']  = numpy.where(team_data['team_is_home'] == True, team_data['FGA'], 0)
    team_data['home_FT']   = numpy.where(team_data['team_is_home'] == True, team_data['FT'], 0)
    team_data['home_FTA']  = numpy.where(team_data['team_is_home'] == True, team_data['FTA'], 0)

    team_data['guest_3FG']  = numpy.where(team_data['team_is_home'] == False, team_data['3FG'], 0)
    team_data['guest_3FGA'] = numpy.where(team_data['team_is_home'] == False, team_data['3FGA'], 0)
    team_data['guest_FGM']  = numpy.where(team_data['team_is_home'] == False, team_data['FGM'], 0)
    team_data['guest_FGA']  = numpy.where(team_data['team_is_home'] == False, team_data['FGA'], 0)
    team_data['guest_FT']   = numpy.where(team_data['team_is_home'] == False, team_data['FT'], 0)
    team_data['guest_FTA']  = numpy.where(team_data['team_is_home'] == False, team_data['FTA'], 0)

    columns = ['game_id',
               'gamedate', 
               'guest', 
               'guest_score', 
               'home', 
               'home_score', 
               'neutral_location', 
               'spread', 
               'guest_id', 
               'home_id',
               'home_3FG',
               'home_3FGA',
               'home_FGM',
               'home_FGA',
               'home_FT',
               'home_FTA',
               'guest_3FG',
               'guest_3FGA',
               'guest_FGM',
               'guest_FGA',
               'guest_FT',
               'guest_FTA'
              ]

    team_data = team_data[columns]
    
    return team_data

In [9]:
teams = {
    'id': list(),
    'name': list()
}

for k, sf in enumerate(schedule_files):
    
    m = re.search('(\d+)-(.*)\.csv', os.path.basename(sf))
    if m is not None:
        team_id, team_name = m.groups()

        teams['id'].append(int(team_id))
        teams['name'].append(team_name)
        
        #print team_id, team_name

teams = pandas.DataFrame.from_dict(teams)

In [10]:
data = None

for k, sf in enumerate(schedule_files):
        
    m = re.search('(\d+)-(.*)\.csv', os.path.basename(sf))
    if m is not None:
        try:
            team_data = format_team_data(pandas.read_csv(sf), teams)            
        except:
            print('--->>> {}'.format(sf))
            x = pandas.read_csv(sf)
            raise
        
        if data is None:
            data = team_data
        else:
            data = data.append(team_data)
            
    if k % 20 == 0:
        print('Finished {} of {}'.format(k, len(schedule_files)))

Finished 0 of 349
Finished 20 of 349
Finished 40 of 349
Finished 60 of 349
Finished 80 of 349
Finished 100 of 349
Finished 120 of 349
Finished 140 of 349
Finished 160 of 349
Finished 180 of 349
Finished 200 of 349
Finished 220 of 349
Finished 240 of 349
Finished 260 of 349
Finished 280 of 349
Finished 300 of 349
Finished 320 of 349
Finished 340 of 349


In [11]:
def match_id(s):
    home_id, guest_id, mm, dd, yyyy = s.split('/')
    return '/'.join([guest_id, home_id, mm, dd, yyyy])

data2 = data[['game_id', 'gamedate']].groupby('game_id').count().reset_index()
data_x = data2.loc[data2['gamedate'] != 2, ['game_id', 'gamedate']]
data_x['match_id'] = data_x['game_id'].apply(match_id)
data_x['game_id_hash'] = data_x['game_id'].apply(hash)
data_x['match_id_hash'] = data_x['match_id'].apply(hash)
data_x['min_hash'] = data_x[['game_id_hash', 'match_id_hash']].min(axis=1)
data_x['game_is_min'] = (data_x['game_id_hash'] == data_x['min_hash'])
data_x['new_id'] = numpy.where(data_x['game_is_min'] == True, data_x['game_id'], data_x['match_id'])

# data_y = data.merge(data_x[['new_id']], left_on='game_id', right_on='new_id', how='left')
# data_y['new_game_id'] = numpy.where(data_y['new_id'].isnull(), data_y['game_id'], data_y['reverse_id'])
# data_y = data_y.drop('game_id', axis=1)
# data_y = data_y.drop('match_id', axis=1)
# data_y = data_y.drop('reverse_id', axis=1)
# data_y['game_id'] = data_y['new_game_id']
# data_y = data_y.drop('new_game_id', axis=1)

bad_games = list(data_x[data_x['game_id'] == data_x['new_id']]['match_id'])
data[data['game_id'].isin(bad_games)]

data[data['game_id'] == '1004/624/03/14/2019']

,game_id,gamedate,guest,guest_score,home,home_score,neutral_location,spread,guest_id,home_id,...,home_FGM,home_FGA,home_FT,home_FTA,guest_3FG,guest_3FGA,guest_FGM,guest_FGA,guest_FT,guest_FTA
28,1004/624/03/14/2019,03/14/2019,Sam Houston St.,71,Central Ark.,76,False,5,624,1004,...,25,56,20,32,0,0,0,0,0,0


In [12]:
q = data[data['game_id'].isin(bad_games)].copy()
q['x_game_id'] = q['game_id'].apply(match_id)

q['x_home'] = q['guest']
q['x_guest'] = q['home']

for feature in ['score', 'id', 'FGM', 'FGA', '3FG', '3FGA', 'FT', 'FTA']:
    q['x_guest_{}'.format(feature)] = q['home_{}'.format(feature)]
    q['x_home_{}'.format(feature)] = q['guest_{}'.format(feature)]

q['x_spread'] = q['x_home_score'] - q['x_guest_score']

q['game_id']= q['x_game_id']
q['guest'] = q['x_guest']
q['home'] = q['x_home']
q['spread'] = q['x_spread']

q = q.drop('x_game_id', axis=1)
q = q.drop('x_home', axis=1)
q = q.drop('x_guest', axis=1)
q = q.drop('x_spread', axis=1)

for feature in ['score', 'id', 'FGM', 'FGA', '3FG', '3FGA', 'FT', 'FTA']:
    q['home_{}'.format(feature)] = q['x_home_{}'.format(feature)] 
    q['guest_{}'.format(feature)] = q['x_guest_{}'.format(feature)] 
    
    q = q.drop('x_home_{}'.format(feature), axis=1)
    q = q.drop('x_guest_{}'.format(feature), axis=1)

data = data[~data['game_id'].isin(bad_games)].append(q)

data

,game_id,gamedate,guest,guest_score,home,home_score,neutral_location,spread,guest_id,home_id,...,home_FGM,home_FGA,home_FT,home_FTA,guest_3FG,guest_3FGA,guest_FGM,guest_FGA,guest_FT,guest_FTA
2,314/1004/11/13/2018,11/13/2018,Central Ark.,56,Jackson St.,65,False,9,1004,314,...,0,0,0,0,2,15,22,60,10,15
3,1004/646/11/17/2018,11/17/2018,South Alabama,58,Central Ark.,50,False,-8,646,1004,...,16,58,15,24,0,0,0,0,0,0
4,17/1004/11/28/2018,11/28/2018,Central Ark.,57,Alcorn,46,False,-11,1004,17,...,0,0,0,0,0,7,19,55,19,34
6,522/1004/12/05/2018,12/05/2018,Central Ark.,52,Oklahoma,65,False,13,1004,522,...,0,0,0,0,2,15,18,58,14,20
7,697/1004/12/08/2018,12/08/2018,Central Ark.,27,Texas A&M,84,False,57,1004,697,...,0,0,0,0,1,11,11,53,4,8
9,327/1004/12/20/2018,12/20/2018,Central Ark.,54,Kansas St.,70,False,16,1004,327,...,0,0,0,0,5,10,19,51,11,12
10,1004/26172/12/31/2018,12/31/2018,A&M-Corpus Christi,73,Central Ark.,65,False,-8,26172,1004,...,22,57,14,17,0,0,0,0,0,0
11,655/1004/01/05/2019,01/05/2019,Central Ark.,53,Southeastern La.,62,False,9,1004,655,...,0,0,0,0,3,11,18,55,14,15
12,624/1004/01/09/2019,01/09/2019,Central Ark.,62,Sam Houston St.,66,False,4,1004,624,...,0,0,0,0,4,9,25,56,8,20
13,346/1004/01/12/2019,01/12/2019,Central Ark.,37,Lamar University,57,False,20,1004,346,...,0,0,0,0,3,15,13,41,8,8


In [13]:
data.shape

(10390, 22)

In [14]:
q = data[['game_id', 'gamedate']].groupby('game_id').count().reset_index()
q[q['gamedate'] != 2][['game_id', 'gamedate']]

,game_id,gamedate


In [15]:
data[data['game_id'] == '702/676/03/21/2019']

,game_id,gamedate,guest,guest_score,home,home_score,neutral_location,spread,guest_id,home_id,...,home_FGM,home_FGA,home_FT,home_FTA,guest_3FG,guest_3FGA,guest_FGM,guest_FGA,guest_FT,guest_FTA
31,702/676/03/21/2019,03/21/2019,SFA,79,UT Arlington,88,False,9,676,702,...,0,0,0,0,9,20,21,45,3,8
30,702/676/03/21/2019,03/21/2019,SFA,54,UT Arlington,60,False,6,676,702,...,23,57,12,12,0,0,0,0,0,0


In [16]:
pandas.options.display.max_columns = 50
data[data['game_id'] == '702/676/03/21/2019'].groupby('game_id').max().reset_index()

,game_id,gamedate,guest,guest_score,home,home_score,neutral_location,spread,guest_id,home_id,home_3FG,home_3FGA,home_FGM,home_FGA,home_FT,home_FTA,guest_3FG,guest_3FGA,guest_FGM,guest_FGA,guest_FT,guest_FTA
0,702/676/03/21/2019,03/21/2019,SFA,79,UT Arlington,88,False,9,676,702,2,16,23,57,12,12,9,20,21,45,3,8


In [17]:
w = data[data['game_id'] == '702/676/03/21/2019'].groupby('game_id').max().reset_index()
list(w['home_score']), list(3*w['home_3FG'] + 2*(w['home_FGM']) + 1*w['home_FT'])

([88], [64])

In [18]:
3*2 + 2*23 + 1*12, 3*9 + 2*21 + 1*3

(64, 72)

In [20]:
final_data = data.groupby('game_id').max().reset_index()

In [45]:
r = final_data.copy()

for feature in ['3FG', '3FGA', 'FGM', 'FGA', 'FT', 'FTA']:
    r['home_{}'.format(feature)] = r['home_{}'.format(feature)].apply(lambda s: int(float(str(s).strip('/'))))
    r['guest_{}'.format(feature)] = r['guest_{}'.format(feature)].apply(lambda s: int(float(str(s).strip('/'))))
    
r['home_consistent'] = (r['home_score'] == 3*r['home_3FG'] + 2*(r['home_FGM'] - r['home_3FG']) + 1*r['home_FT'])
r['guest_consistent'] = (r['guest_score'] == 3*r['guest_3FG'] + 2*(r['guest_FGM'] - r['guest_3FG']) + 1*r['guest_FT'])

r[r['home_consistent'] == False].shape, r[r['guest_consistent'] == False].shape 

((3, 24), (4, 24))

In [47]:
r[r['home_consistent'] == False]

,game_id,gamedate,guest,guest_score,home,home_score,neutral_location,spread,guest_id,home_id,home_3FG,home_3FGA,home_FGM,home_FGA,home_FT,home_FTA,guest_3FG,guest_3FGA,guest_FGM,guest_FGA,guest_FT,guest_FTA,home_consistent,guest_consistent
3035,519/86/03/16/2019,03/16/2019,Buffalo,77,Ohio,61,True,-16,86,519,0,0,0,0,0,0,0,0,0,0,0,0,False,False
4334,702/676/03/21/2019,03/21/2019,SFA,79,UT Arlington,88,False,9,676,702,2,16,23,57,12,12,9,20,21,45,3,8,False,False
4584,735/71/12/21/2018,12/21/2018,Bowling Green,79,Valparaiso,72,False,-7,71,735,17,38,24,54,5,14,14,30,26,54,13,18,False,True


In [48]:
r[r['guest_consistent'] == False]

,game_id,gamedate,guest,guest_score,home,home_score,neutral_location,spread,guest_id,home_id,home_3FG,home_3FGA,home_FGM,home_FGA,home_FT,home_FTA,guest_3FG,guest_3FGA,guest_FGM,guest_FGA,guest_FT,guest_FTA,home_consistent,guest_consistent
3035,519/86/03/16/2019,03/16/2019,Buffalo,77,Ohio,61,True,-16,86,519,0,0,0,0,0,0,0,0,0,0,0,0,False,False
4078,676/26172/03/16/2019,03/16/2019,A&M-Corpus Christi,58,SFA,56,True,-2,26172,676,4,19,24,55,4,9,2,8,42,20,16,21,True,False
4334,702/676/03/21/2019,03/21/2019,SFA,79,UT Arlington,88,False,9,676,702,2,16,23,57,12,12,9,20,21,45,3,8,False,False
5121,9/458/03/14/2019,03/14/2019,Charlotte,55,UAB,70,True,15,458,9,10,17,24,53,12,15,3,14,18,55,14,20,True,False


In [53]:
z = r[(r['guest_consistent'] == True) & (r['guest_consistent'] == True)]
output_path = 'final_team_data.csv'
z.to_csv(output_path, index=False)